In [2]:
from scipy.io import loadmat
import numpy as np
from InFusionTools import convertTime
import os

In [3]:
fn = r"C:\Users\broll\ZF Friedrichshafen AG\33658 InFusion - Documents\Grunddatenerhebung\07 Winterversuche\Data\Extracted\Rekorder_2022_02_05_11_06_30.mat"
f = loadmat(fn)

In [3]:
from pymongo import MongoClient

# Connect to the MongoDB on localHost
#TODO: Create MongoDB on ZF-Server
client = MongoClient("127.0.0.1", port=27017)

# Check if "InFusion"-Database already exists, else create the database "InFusion"
dblist = client.list_database_names()
if "InFusion" in dblist:
    print("The database exists.")
    db = client.InFusion
else: 
    db = client["InFusion"]

""" # debugging
# Check if "test"-Collection already exists, else create the collection "test"
#TODO replace collection with meaningfull collection for InFusion data
collist = db.list_collection_names()
if col in collist:
    print("The collection exists.")
    col = db[col]
else: 
    col = db[col] """

The database exists.


' # debugging\n# Check if "test"-Collection already exists, else create the collection "test"\n#TODO replace collection with meaningfull collection for InFusion data\ncollist = db.list_collection_names()\nif col in collist:\n    print("The collection exists.")\n    col = db[col]\nelse: \n    col = db[col] '

In [ ]:
fn = r"C:\Users\broll\ZF Friedrichshafen AG\33658 InFusion - Documents\Grunddatenerhebung\07 Winterversuche\Data\Extracted"
l_fileNames = os.listdir(fn)
for f in l_fileNames:
    print(os.path.join(fn, f))

In [2]:
#folderName = r"C:\Users\broll\ZF Friedrichshafen AG\33658 InFusion - Documents\Grunddatenerhebung\07 Winterversuche\Data\Extracted"
folderName = r"C:\Users\broll\ZF Friedrichshafen AG\33658 InFusion - Documents\Grunddatenerhebung\04 Data\20210802\extracted"

l_fileNames = os.listdir(folderName)
convertChannelWL = []
convertMessagesWL = []
for fileName in l_fileNames:
    print("Loading: ", fileName)
    fn = os.path.join(folderName, fileName)
    f = loadmat(fn)

    f_name, f_ext = os.path.splitext(fn)
    s_FileName = f_name.split("\\")[-1]
    t0 = convertTime(s_FileName)
    errorMessages = []

    for i, channel in enumerate(f.keys()):
        if convertChannelWL and not channel in convertChannelWL:
            continue
        #print("#1", channel)
        if channel in ["__header__", "__version__", "__globals__"]:
            #print("#2", channel)
            continue
        #print("#3", channel)
        for ii, message in enumerate(f[channel][0][0].dtype.fields.keys()):
            if convertMessagesWL and not message in convertMessagesWL:
                continue
            # TODO: Fix Failing messages:
            if message == "FMS": 
                continue
            #print("#4", ii, message)
            if message == "SoundAI": #SoundAI is a single Sensor and does not have multiple signals
                l_timeStamps, l_values = [list(x) for x in zip(*f[channel][0][0][message])]
                l_signalNames = ["SoundAI"]
            else:
                l_signalNames = []
                l_values = []
                for iii, signal in enumerate(f[channel][0][0][message].dtype.fields.keys()):
                    # get two lists, first containing the names of signals as string [<signal1>, <signal2>, ...] 
                    #   second containing [[timestamps (as ID)], [<signal1_values>], [<signal2_values>], ...]
                    l_signalNames.append(signal)
                    if iii == 0: # extract timestamps once, as they are the same for all signals in a message
                        l_timeStamps, temp = [list(x) for x in zip(*f[channel][0][0][message][signal][0][0])]
                        l_values.append(temp)
                    else:
                        #print("#5", iii, signal)
                        _, temp = [list(x) for x in zip(*f[channel][0][0][message][signal][0][0])]
                        l_values.append(temp)
                
            # combine timestamps from measurement with start time of measurement to create identifier in database
            l_timeStamps = [np.round((t0 + td) * 100000) for td in l_timeStamps]
            collist = db.list_collection_names()
            if message in collist:
                #print("The collection exists.")
                col = db[message]
            else: 
                col = db[message]
            
            for iv in range(len(l_timeStamps)):
                upload = {"_id": l_timeStamps[iv]}
                if message == "SoundAI":
                    sig = "SoundAI"
                    upload.update({sig: l_values[iv]})
                else:
                    for v, sig in enumerate(l_signalNames):
                        #print("#6", sig)
                        try:
                            upload.update({sig: l_values[v][iv]})
                        except:
                            if not message in errorMessages:
                                errorMessages.append(message)
                                print("E1: An exception occurred", channel, message,  sig, v, iv)
                try:
                    col.insert_one( upload )
                except:
                    if not message in errorMessages:
                        errorMessages.append(message)
                        print("E2: An exception occurred", channel, message,  sig, v, iv)
                #print("done")
        

Loading:  Recorder_2021_08_02_08_14_23.mat


ValueError: time data 'Recorder_2021_08_02_08_14_23' does not match format 'Rekorder_%Y_%m_%d_%H_%M_%S'